# Master Thesis


In [1]:
from utils import *
from network import *
import pandas as pd
import importlib

## Bipartite album-artists network 

In [2]:
net = Network()
utils = Utils("./config.ini")
df,albums_json=utils.load_data_top_5000_albums()
bipartite_G,nb_album = net.build_bipartite_network(df,albums_json)

In [4]:
nb_album

4394

In [3]:
net.get_network_info(bipartite_G)

nb nodes:  26545
nb edges:  53047
Min node degree:  1
Max node degree:  520
Avg node degree:  3.9967602184968922


In [4]:
top50musicians = sorted(bipartite_G.degree, key=lambda x: x[1], reverse=False)[0]

In [5]:
top50musicians

(5725699, 0)